# Homework 2

**Name:** -- Agustin Villarreal --

**e-mail:** -- agustin.villarreal0743@alumnos.udg.mx --

# MODULES

In [2]:
# Import necessary libraries
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random
import os

# Activity 1: Path Length Comparison - Brownian Motion vs Correlated Random Walk

In this activity, we'll compare the path lengths of different types of random walks:
Brownian Motion (BM) and Correlated Random Walk (CRW). We'll generate trajectories 
for each type of movement, calculate their path lengths, and visualize the results.


In [13]:
class Vector2D:
    """A two-dimensional vector class with operations for trajectory generation."""
    
    def __init__(self, x, y=None):
        """Initialize a 2D vector from coordinates or an iterable."""
        if y is None and hasattr(x, "__getitem__"):
            self.x = x[0]
            self.y = x[1]
        else:
            self.x = x
            self.y = y
    
    def __add__(self, other):
        """Vector addition."""
        if isinstance(other, Vector2D):
            return Vector2D(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vector2D(self.x + other[0], self.y + other[1])
        else:
            return Vector2D(self.x + other, self.y + other)
    
    def __sub__(self, other):
        """Vector subtraction."""
        if isinstance(other, Vector2D):
            return Vector2D(self.x - other.x, self.y - other.y)
        elif hasattr(other, "__getitem__"):
            return Vector2D(self.x - other[0], self.y - other[1])
        else:
            return Vector2D(self.x - other, self.y - other)
    
    def length(self):
        """Calculate the vector magnitude."""
        return math.sqrt(self.x**2 + self.y**2)
    
    def rotate(self, angle):
        """Rotate the vector by angle (in radians)."""
        cos_a = math.cos(angle)
        sin_a = math.sin(angle)
        return Vector2D(
            self.x * cos_a - self.y * sin_a, 
            self.x * sin_a + self.y * cos_a
        )


## Brownian Motion Implementation

Brownian Motion (BM) is characterized by complete randomness in direction at each step.
The mathematical model for 2D Brownian Motion can be described as:

$$x_{t+1} = x_t + v cos(θ_t)$$
$$y_{t+1} = y_t + v sin(θ_t)$$

Where:
- $(x_t, y_t)$ is the position at time $t$
- $v$ is the constant speed
- $θ_t$ is a random angle uniformly distributed between $\pi$ and $\pi$

In [14]:
def generate_brownian_motion(n_steps=1000, speed=5, start_pos=(0, 0)):
    """
    Generate a 2D Brownian Motion trajectory.
    
    Args:
        n_steps: Number of steps in the trajectory
        speed: Step size (constant)
        start_pos: Starting position as (x, y) tuple
        
    Returns:
        pandas.DataFrame with columns 'x' and 'y' representing positions
    """
    positions = [start_pos]
    velocity = Vector2D(speed, 0)  # Initial velocity vector
    
    for _ in range(n_steps - 1):
        # In Brownian Motion, direction is completely random
        angle = np.random.uniform(-np.pi, np.pi)
        velocity = velocity.rotate(angle)
        
        # Calculate new position
        prev_x, prev_y = positions[-1]
        new_pos = (prev_x + velocity.x, prev_y + velocity.y)
        positions.append(new_pos)
    
    # Convert to DataFrame
    return pd.DataFrame(positions, columns=['x', 'y'])

# Correlated Random Walk Implementation

Correlated Random Walk (CRW) introduces directional persistence, where the new direction
is a slight deviation from the previous direction. The mathematical model can be expressed as:

$$x_{t+1} = x_t + v cos(φ_t)$$
$$y_{t+1} = y_t + v sin(φ_t)$$
$$φ_t = φ_{t-1} + ε_t$$

Where:
- $(x_t, y_t)$ is the position at time $t$
- $v$ is the constant speed
- $φ_t$ is the heading angle at time $t$
- $ε_t$ is a random turning angle drawn from a normal distribution $N(0, σ²)$

In [15]:
def generate_correlated_random_walk(n_steps=1000, speed=5, start_pos=(0, 0), sigma=np.pi/8):
    """
    Generate a 2D Correlated Random Walk trajectory.
    
    Args:
        n_steps: Number of steps in the trajectory
        speed: Step size (constant)
        start_pos: Starting position as (x, y) tuple
        sigma: Standard deviation for the turning angle distribution
        
    Returns:
        pandas.DataFrame with columns 'x' and 'y' representing positions
    """
    positions = [start_pos]
    current_angle = 0  # Initial heading direction
    
    for _ in range(n_steps - 1):
        # In CRW, direction changes are normally distributed around the previous direction
        turn_angle = np.random.normal(0, sigma)
        current_angle += turn_angle
        
        # Calculate new position
        prev_x, prev_y = positions[-1]
        new_x = prev_x + speed * math.cos(current_angle)
        new_y = prev_y + speed * math.sin(current_angle)
        positions.append((new_x, new_y))
    
    # Convert to DataFrame
    return pd.DataFrame(positions, columns=['x', 'y'])


## Path Length Calculation

The path length of a trajectory is the sum of the distances between consecutive points.
For a trajectory with n points, the path length is:

$$L = Σ√[(x_{i+1} - x_i)² + (y_{i+1} - y_i)²]$$

This function calculates the total distance traveled along the path.

In [16]:
def calculate_path_length(trajectory):
    """
    Calculate the total path length of a trajectory.
    
    Args:
        trajectory: pandas.DataFrame with 'x' and 'y' columns
        
    Returns:
        float: Total path length
    """
    # Calculate differences between consecutive positions
    diffs = trajectory.diff().dropna()
    
    # Calculate the length of each step
    step_lengths = np.sqrt(diffs['x']**2 + diffs['y']**2)
    
    # Return the sum of all step lengths
    return step_lengths.sum()

## Cumulative Path Length

The cumulative path length at step j is the sum of the distances up to that point:

$$L_j = Σ√[(x_{i+1} - x_i)² + (y_{i+1} - y_i)²] \text{for i=1 to j}$$

This allows us to see how the total distance traveled increases over time.

In [ ]:
def calculate_cumulative_path_length(trajectory):
    """
    Calculate the cumulative path length at each step.
    
    Args:
        trajectory: pandas.DataFrame with 'x' and 'y' columns
        
    Returns:
        numpy.ndarray: Cumulative path lengths
    """
    # Calculate differences between consecutive positions
    diffs = trajectory.diff().dropna()
    
    # Calculate the length of each step
    step_lengths = np.sqrt(diffs['x']**2 + diffs['y']**2)
    
    # Calculate cumulative sum
    return np.array(np.cumsum(step_lengths))

In [22]:
# Set random seed for reproducibility
np.random.seed(42)

# Set parameters
n_steps = 1000
start_pos = (0, 0)

# Define trajectory configurations
trajectories_config = {
    'BM 3': {'type': 'BM', 'speed': 3, 'color': 'royalblue'},
    'BM 6': {'type': 'BM', 'speed': 6, 'color': 'red', 'dash': 'dash'},
    'CRW 6': {'type': 'CRW', 'speed': 6, 'sigma': np.pi/8, 'color': 'seagreen'},
    # Add more trajectories here by adding new dictionary entries
    'BM 9': {'type': 'BM', 'speed': 9, 'color': 'purple'},
    'CRW 18': {'type': 'CRW', 'speed': 18, 'sigma': np.pi/8, 'color': 'black'},
}

# Generate trajectories and calculate path lengths
trajectory_data = {}
cum_lengths = {}
total_lengths = {}

for name, config in trajectories_config.items():
    # Generate appropriate trajectory
    if config['type'] == 'BM':
        trajectory_data[name] = generate_brownian_motion(
            n_steps=n_steps, 
            speed=config['speed'], 
            start_pos=start_pos
        )
    else:  # CRW
        trajectory_data[name] = generate_correlated_random_walk(
            n_steps=n_steps, 
            speed=config['speed'], 
            start_pos=start_pos, 
            sigma=config['sigma']
        )
    
    # Calculate path lengths
    cum_lengths[name] = calculate_cumulative_path_length(trajectory_data[name])
    total_lengths[name] = cum_lengths[name][-1] if isinstance(cum_lengths[name], np.ndarray) else cum_lengths[name].iloc[-1]
    
    print(f"Total path length for {name}: {total_lengths[name]:.2f}")

# Create plot
fig = go.Figure()

# Add traces for each trajectory
for name, cum_length in cum_lengths.items():
    fig.add_trace(go.Scatter(
        x=list(range(len(cum_length))),
        y=cum_length,
        mode='lines',
        name=f'path length {name}',
        line=dict(
            color=trajectories_config[name]['color'],
            dash=trajectories_config[name].get('dash'),
            width=3  # Thicker lines for better visibility
        )
    ))

# Update layout
fig.update_layout(
    title='Comparison of Path Lengths',
    xaxis_title='Step Number',
    yaxis_title='Cumulative Path Length',
    template='plotly_white',
    plot_bgcolor='rgba(240, 240, 250, 0.8)',
    legend=dict(
        x=1.02,
        y=0.98,
        xanchor='left',
        yanchor='top'
    ),
    width=800,
    height=500
)

# Add grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the plot
fig.show()

Total path length for BM 3: 2997.00
Total path length for BM 6: 5994.00
Total path length for CRW 6: 5994.00
Total path length for BM 9: 8991.00
Total path length for CRW 18: 17982.00


## Module 5: Multiple Correlated Random Walks

This module extends the single CRW model to simulate multiple walkers with different correlation parameters. This allows for comparative analysis of how the correlation strength affects the trajectory characteristics.

Each walker follows the same CRW model as described in Module 1, but with a different Cauchy coefficient $c_i$ for the turning angle distribution:

$$\theta_{i,t+1} \sim \text{Cauchy}(0, c_i)$$

By visualizing multiple trajectories simultaneously, we can observe the spectrum of behaviors from highly correlated (nearly straight-line) movement to nearly uncorrelated (Brownian-like) movement.

This comparative approach is useful for:

1. Studying the transition from ballistic motion (MSD $\propto \tau^2$) at short time scales to diffusive motion (MSD $\propto \tau$) at long time scales, and how this transition depends on the correlation strength.
2. Understanding how the persistence length (the characteristic distance over which directional correlations decay) relates to the Cauchy coefficient.
3. Modeling diverse movement strategies in heterogeneous populations, such as different individuals in animal groups or cells in tissue.

Multiple CRW visualization demonstrates how small changes in the correlation parameter can lead to significantly different spatial patterns and exploration efficiencies, highlighting the importance of directional persistence in movement ecology and related fields.

In [17]:
# Module 5: Multiple CRW Trajectories
def generate_multiple_crw_trajectories(num_trajectories, num_steps, speed, 
                                      start_pos_x, start_pos_y, 
                                      cauchy_coefs, seed=None):
    """
    Generate multiple CRW trajectories with different Cauchy coefficients
    
    Parameters:
    -----------
    num_trajectories : int
        Number of trajectories to generate
    num_steps : int
        Number of steps in each trajectory
    speed : float
        Speed/step size for the walkers
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coefs : list or array
        Cauchy coefficients for each trajectory
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectories : list
        List of trajectory arrays
    """
    if seed is not None:
        np.random.seed(seed)
        
    if len(cauchy_coefs) < num_trajectories:
        raise ValueError("Not enough Cauchy coefficients provided")
    
    trajectories = []
    
    for i in range(num_trajectories):
        traj = generate_crw_trajectory(
            num_steps=num_steps,
            speed=speed,
            start_pos_x=start_pos_x,
            start_pos_y=start_pos_y,
            cauchy_coef=cauchy_coefs[i],
            seed=seed+i if seed is not None else None
        )
        trajectories.append(traj)
    
    return trajectories


Mean Squared Displacement (MSD)

A key analytical tool for characterizing random walks is the Mean Squared Displacement (MSD), which measures how far the walker travels from its starting point over time. For a temporal lag $\tau$, the MSD is defined as:

$$\text{MSD}(\tau) = \langle |\vec{r}_{t+\tau} - \vec{r}_t|^2 \rangle_t$$

where $\langle \cdot \rangle_t$ denotes an average over all possible starting times $t$.

For a standard random walk, the MSD scales linearly with time: $\text{MSD}(\tau) \propto \tau$, indicating diffusive behavior. For a CRW, the MSD initially scales super-diffusively (faster than linear) at short time scales, but eventually transitions to diffusive behavior at longer time scales. This transition point depends on the correlation strength.

In [9]:
# Calculate Mean Squared Displacement (MSD)
def calculate_msd(trajectory):
    """
    Calculate Mean Squared Displacement for a trajectory
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y coordinates
        
    Returns:
    --------
    tau : numpy.ndarray
        Time lags
    msd : numpy.ndarray
        MSD values for each time lag
    """
    positions = trajectory[:, :2]  # Only x, y coordinates
    n_points = len(positions)
    
    # Calculate maximum tau (time lag)
    max_tau = n_points // 4  # Use 1/4 of the trajectory for reliable statistics
    
    tau = np.arange(1, max_tau + 1)
    msd = np.zeros(max_tau)
    
    # Calculate MSD for each tau
    for i, t in enumerate(tau):
        # Calculate squared displacements
        sd = np.sum((positions[t:] - positions[:-t])**2, axis=1)
        # Average to get MSD
        msd[i] = np.mean(sd)
    
    return tau, msd

def plot_msd(tau, msd, title="Mean Squared Displacement"):
    """
    Plot Mean Squared Displacement
    
    Parameters:
    -----------
    tau : array_like
        Time lags
    msd : array_like
        MSD values
    title : str
        Plot title
        
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add MSD trace
    fig.add_trace(go.Scatter(
        x=tau,
        y=msd,
        mode='lines',
        name='MSD',
        line=dict(color='purple', width=2)
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="τ",
        yaxis_title="MSD",
        template="plotly_white"
    )
    
    return fig

In [10]:
def create_dash_dashboard():
    """Create a Dash dashboard for random walks visualization"""
    
    # Import standard Dash instead of JupyterDash
    import dash
    from dash import dcc, html
    from dash.dependencies import Input, Output, State
    import plotly.graph_objs as go
    import dash_bootstrap_components as dbc
    import numpy as np
    
    # Initialize the standard Dash app
    app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
    
    # App layout
    app.layout = html.Div([
        html.H1("Random Walks Dashboard", className="text-center my-4"),
        
        # Tabs for different activities
        dbc.Tabs([
            # Activity 1: CRW - 1 Trajectory
            dbc.Tab(label="Activity 1: CRW", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("Number of steps"),
                        dcc.Input(
                            id="crw-num-steps",
                            type="number",
                            value=300,  # Default to a smaller value for performance
                            min=100,
                            max=5000,
                            step=100,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Speed"),
                        dcc.Input(
                            id="crw-speed",
                            type="number",
                            value=5,
                            min=1,
                            max=20,
                            step=1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_x"),
                        dcc.Input(
                            id="crw-start-x",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_y"),
                        dcc.Input(
                            id="crw-start-y",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Cauchy coefficient"),
                        dcc.Input(
                            id="crw-cauchy",
                            type="number",
                            value=0.7,
                            min=0.1,
                            max=0.9,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="crw-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Trajectory plot
                    dbc.Col([
                        html.H5("Trajectory", className="mt-3"),
                        dcc.Graph(
                            id="crw-trajectory-graph",
                            style={"height": "500px"}
                        )
                    ], width=5),
                    
                    # MSD plot
                    dbc.Col([
                        html.H5("Metric", className="mt-3"),
                        dcc.Dropdown(
                            id="crw-metric-type",
                            options=[
                                {"label": "MSD", "value": "MSD"}
                            ],
                            value="MSD",
                            clearable=False,
                            className="mb-2"
                        ),
                        dcc.Graph(
                            id="crw-msd-graph",
                            style={"height": "500px"}
                        )
                    ], width=4)
                ])
            ]),
            
            # Activity 2: Lévy distribution - N different curves
            dbc.Tab(label="Activity 2: Lévy Curves", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("μ parameter"),
                        dcc.Input(
                            id="levy-curves-mu",
                            type="number",
                            value=3.0,
                            min=0,
                            max=10,
                            step=0.5,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("β parameter"),
                        dcc.Input(
                            id="levy-curves-beta",
                            type="number",
                            value=0,
                            min=-1,
                            max=1,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.H6("Alpha Values", className="mt-3"),
                        dbc.Checklist(
                            id="levy-curves-alphas",
                            options=[
                                {"label": "α = 0.1", "value": 0.1},
                                {"label": "α = 0.5", "value": 0.5},
                                {"label": "α = 1.0", "value": 1.0},
                                {"label": "α = 1.9", "value": 1.9}
                            ],
                            value=[0.1, 0.5, 1.0, 1.9],  # Default all selected
                            className="mb-3"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="levy-curves-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Lévy distributions plot
                    dbc.Col([
                        html.H5("Lévy Distributions", className="mt-3"),
                        dcc.Graph(
                            id="levy-curves-graph",
                            style={"height": "600px"}
                        )
                    ], width=9)
                ])
            ]),
            
            # Activity 3: Lévy distribution - Histogram + Curve
            dbc.Tab(label="Activity 3: Lévy Histogram", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("α parameter"),
                        dcc.Input(
                            id="levy-hist-alpha",
                            type="number",
                            value=1.5,
                            min=0.1,
                            max=2.0,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("β parameter"),
                        dcc.Input(
                            id="levy-hist-beta",
                            type="number",
                            value=0,
                            min=-1,
                            max=1,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("μ parameter"),
                        dcc.Input(
                            id="levy-hist-mu",
                            type="number",
                            value=3.0,
                            min=0,
                            max=10,
                            step=0.5,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("c parameter"),
                        dcc.Input(
                            id="levy-hist-c",
                            type="number",
                            value=1.0,
                            min=0.1,
                            max=5.0,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("# Samples"),
                        dcc.Input(
                            id="levy-hist-samples",
                            type="number",
                            value=1000,
                            min=100,
                            max=10000,
                            step=100,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("# Bins"),
                        dcc.Input(
                            id="levy-hist-bins",
                            type="number",
                            value=50,
                            min=10,
                            max=200,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="levy-hist-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Histogram plot
                    dbc.Col([
                        html.H5("Lévy Distribution Histogram", className="mt-3"),
                        dcc.Graph(
                            id="levy-hist-graph",
                            style={"height": "600px"}
                        )
                    ], width=9)
                ])
            ]),
            
            # Activity 4: Lévy flight - Vec2d - 1 Trajectory
            dbc.Tab(label="Activity 4: Lévy Flight", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("Number of steps"),
                        dcc.Input(
                            id="levy-flight-steps",
                            type="number",
                            value=300,
                            min=100,
                            max=5000,
                            step=100,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("α parameter"),
                        dcc.Input(
                            id="levy-flight-alpha",
                            type="number",
                            value=1.5,
                            min=0.1,
                            max=2.0,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("μ parameter"),
                        dcc.Input(
                            id="levy-flight-mu",
                            type="number",
                            value=3.0,
                            min=0,
                            max=10,
                            step=0.5,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("c parameter"),
                        dcc.Input(
                            id="levy-flight-c",
                            type="number",
                            value=1.0,
                            min=0.1,
                            max=5.0,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_x"),
                        dcc.Input(
                            id="levy-flight-start-x",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_y"),
                        dcc.Input(
                            id="levy-flight-start-y",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Cauchy coefficient"),
                        dcc.Input(
                            id="levy-flight-cauchy",
                            type="number",
                            value=0.7,
                            min=0.1,
                            max=0.9,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="levy-flight-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Trajectory plot
                    dbc.Col([
                        html.H5("Lévy Flight Trajectory", className="mt-3"),
                        dcc.Graph(
                            id="levy-flight-graph",
                            style={"height": "600px"}
                        )
                    ], width=9)
                ])
            ]),
            
            # Activity 5: Correlated Random Walk - Vec2d - N Trajectories
            dbc.Tab(label="Activity 5: Multiple CRW", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("Number of trajectories"),
                        dcc.Input(
                            id="multi-crw-n-traj",
                            type="number",
                            value=6,
                            min=2,
                            max=10,
                            step=1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Steps per trajectory"),
                        dcc.Input(
                            id="multi-crw-steps",
                            type="number",
                            value=300,
                            min=100,
                            max=5000,
                            step=100,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Speed"),
                        dcc.Input(
                            id="multi-crw-speed",
                            type="number",
                            value=5,
                            min=1,
                            max=20,
                            step=1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_x"),
                        dcc.Input(
                            id="multi-crw-start-x",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_y"),
                        dcc.Input(
                            id="multi-crw-start-y",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="multi-crw-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Multiple trajectories plot
                    dbc.Col([
                        html.H5("Multiple CRW Trajectories", className="mt-3"),
                        dcc.Graph(
                            id="multi-crw-graph",
                            style={"height": "600px"}
                        )
                    ], width=9)
                ])
            ])
        ], className="mb-3")
    ], className="container-fluid")
    
    # Activity 1: CRW - 1 Trajectory - Callbacks
    @app.callback(
        [Output("crw-trajectory-graph", "figure"),
         Output("crw-msd-graph", "figure")],
        Input("crw-update-button", "n_clicks"),
        [State("crw-num-steps", "value"),
         State("crw-speed", "value"),
         State("crw-start-x", "value"),
         State("crw-start-y", "value"),
         State("crw-cauchy", "value")]
    )
    def update_crw(n_clicks, num_steps, speed, start_x, start_y, cauchy_coef):
        # Generate trajectory
        trajectory = generate_crw_trajectory(
            num_steps=num_steps,
            speed=speed,
            start_pos_x=start_x,
            start_pos_y=start_y,
            cauchy_coef=cauchy_coef
        )
        
        # Calculate MSD
        tau, msd = calculate_msd(trajectory)
        
        # Plot every nth point to improve performance
        plot_every_n = max(1, len(trajectory) // 500)
        
        # Create trajectory plot
        traj_fig = go.Figure()
        
        traj_fig.add_trace(go.Scatter3d(
            x=trajectory[::plot_every_n, 0],
            y=trajectory[::plot_every_n, 1],
            z=trajectory[::plot_every_n, 2],
            mode='lines',
            name='CRW',
            line=dict(color='red', width=3)
        ))
        
        traj_fig.update_layout(
            title="CRW trajectory",
            scene=dict(
                xaxis_title="x_pos",
                yaxis_title="y_pos",
                zaxis_title="time",
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=1)
            ),
            margin=dict(l=0, r=0, b=0, t=30),
            template="plotly_white"
        )
        
        # Create MSD plot
        msd_fig = go.Figure()
        
        msd_fig.add_trace(go.Scatter(
            x=tau,
            y=msd,
            mode='lines',
            name='MSD',
            line=dict(color='purple', width=2)
        ))
        
        msd_fig.update_layout(
            title="Mean Squared Displacement",
            xaxis_title="tau",
            yaxis_title="MSD",
            template="plotly_white",
            margin=dict(l=0, r=0, b=0, t=30)
        )
        
        return traj_fig, msd_fig
    
    # Activity 2: Lévy distribution - N different curves - Callbacks
    @app.callback(
        Output("levy-curves-graph", "figure"),
        Input("levy-curves-update-button", "n_clicks"),
        [State("levy-curves-mu", "value"),
         State("levy-curves-beta", "value"),
         State("levy-curves-alphas", "value")]
    )
    def update_levy_curves(n_clicks, mu, beta, alphas):
        if not alphas:
            # Default to one alpha if none selected
            alphas = [1.0]
        
        # Generate x range
        x_range = np.linspace(-2, 8, 1000)
        
        # Create figure
        fig = go.Figure()
        
        # Add a trace for each alpha value
        for alpha in alphas:
            pdf_values = levy_pdf(x_range, alpha, beta, mu)
            
            fig.add_trace(go.Scatter(
                x=x_range,
                y=pdf_values,
                mode='lines',
                name=f'Lévy {alpha}',
                line=dict(width=2)
            ))
        
        # Update layout
        fig.update_layout(
            title="Lévy Stable Distributions",
            xaxis_title="x",
            yaxis_title="Probability Density",
            template="plotly_white",
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99
            )
        )
        
        return fig
    
    # Activity 3: Lévy distribution - Histogram + Curve - Callbacks
    @app.callback(
        Output("levy-hist-graph", "figure"),
        Input("levy-hist-update-button", "n_clicks"),
        [State("levy-hist-alpha", "value"),
         State("levy-hist-beta", "value"),
         State("levy-hist-mu", "value"),
         State("levy-hist-c", "value"),
         State("levy-hist-samples", "value"),
         State("levy-hist-bins", "value")]
    )
    def update_levy_histogram(n_clicks, alpha, beta, mu, c, n_samples, n_bins):
        # Generate samples
        samples = generate_levy_samples(
            n_samples=n_samples,
            alpha=alpha,
            beta=beta,
            mu=mu,
            c=c
        )
        
        # Create x range for PDF
        x_min, x_max = np.min(samples), np.max(samples)
        x_range = np.linspace(x_min, x_max, 1000)
        
        # Calculate PDF
        pdf_values = levy_pdf(x_range, alpha, beta, mu, c)
        
        # Create figure
        fig = go.Figure()
        
        # Add histogram
        fig.add_trace(go.Histogram(
            x=samples,
            histnorm='probability density',
            name='Samples',
            nbinsx=n_bins,
            opacity=0.5,
            marker_color='skyblue'
        ))
        
        # Add PDF curve
        fig.add_trace(go.Scatter(
            x=x_range,
            y=pdf_values,
            mode='lines',
            name='PDF',
            line=dict(color='red', width=2)
        ))
        
        # Update layout
        fig.update_layout(
            title=f"Lévy Distribution (α={alpha}, β={beta}, μ={mu})",
            xaxis_title="x",
            yaxis_title="Probability Density",
            template="plotly_white",
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99
            )
        )
        
        return fig
    
    # Activity 4: Lévy flight - Vec2d - 1 Trajectory - Callbacks
    @app.callback(
        Output("levy-flight-graph", "figure"),
        Input("levy-flight-update-button", "n_clicks"),
        [State("levy-flight-steps", "value"),
         State("levy-flight-alpha", "value"),
         State("levy-flight-mu", "value"),
         State("levy-flight-c", "value"),
         State("levy-flight-start-x", "value"),
         State("levy-flight-start-y", "value"),
         State("levy-flight-cauchy", "value")]
    )
    def update_levy_flight(n_clicks, num_steps, alpha, mu, c, start_x, start_y, cauchy_coef):
        # Generate trajectory
        trajectory = generate_levy_flight_trajectory(
            num_steps=num_steps,
            alpha=alpha,
            mu=mu,
            c=c,
            start_pos_x=start_x,
            start_pos_y=start_y,
            cauchy_coef=cauchy_coef
        )
        
        # Plot every nth point to improve performance
        plot_every_n = max(1, len(trajectory) // 500)
        
        # Create figure
        fig = go.Figure()
        
        fig.add_trace(go.Scatter3d(
            x=trajectory[::plot_every_n, 0],
            y=trajectory[::plot_every_n, 1],
            z=trajectory[::plot_every_n, 2],
            mode='lines',
            name='Lévy Flight',
            line=dict(color='blue', width=3)
        ))
        
        fig.update_layout(
            title="Lévy Flight 3D",
            scene=dict(
                xaxis_title="x_pos",
                yaxis_title="y_pos",
                zaxis_title="time",
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=1)
            ),
            margin=dict(l=0, r=0, b=0, t=30),
            template="plotly_white"
        )
        
        return fig
    
    # Activity 5: Correlated Random Walk - Vec2d - N Trajectories - Callbacks
    @app.callback(
        Output("multi-crw-graph", "figure"),
        Input("multi-crw-update-button", "n_clicks"),
        [State("multi-crw-n-traj", "value"),
         State("multi-crw-steps", "value"),
         State("multi-crw-speed", "value"),
         State("multi-crw-start-x", "value"),
         State("multi-crw-start-y", "value")]
    )
    def update_multi_crw(n_clicks, n_traj, num_steps, speed, start_x, start_y):
        # Generate Cauchy coefficients
        cauchy_coefs = np.linspace(0.2, 0.95, n_traj)
        
        # Generate trajectories
        trajectories = generate_multiple_crw_trajectories(
            num_trajectories=n_traj,
            num_steps=num_steps,
            speed=speed,
            start_pos_x=start_x,
            start_pos_y=start_y,
            cauchy_coefs=cauchy_coefs
        )
        
        # Create color scale
        colors = [
            'blue', 'red', 'green', 'purple', 'orange', 
            'cyan', 'magenta', 'yellow', 'pink', 'brown'
        ]
        
        # Create figure
        fig = go.Figure()
        
        # Add each trajectory
        for i, (traj, coef) in enumerate(zip(trajectories, cauchy_coefs)):
            color_idx = i % len(colors)
            
            # Plot every nth point to improve performance
            plot_every_n = max(1, len(traj) // 300)
            
            fig.add_trace(go.Scatter3d(
                x=traj[::plot_every_n, 0],
                y=traj[::plot_every_n, 1],
                z=traj[::plot_every_n, 2],
                mode='lines',
                name=f'CRW, Cauchy {coef:.2f}',
                line=dict(color=colors[color_idx], width=3)
            ))
        
        fig.update_layout(
            title="N Correlated Random Walks",
            scene=dict(
                xaxis_title="x_pos",
                yaxis_title="y_pos",
                zaxis_title="time",
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=1)
            ),
            margin=dict(l=0, r=0, b=0, t=30),
            template="plotly_white"
        )
        
        return fig
    
    # Display introduction and launch the dashboard
    print("Random Walks Dashboard is being created...")
    print("Use the 'Update' buttons on each tab to generate visualizations.")
    
    # Return the app for running
    return app

In [11]:
def run_dashboard():
    """Run the dashboard using standard Dash"""
    
    # Create the dashboard app
    app = create_dash_dashboard()
    
    # Run the app in a browser tab
    print("Dashboard will open in a new browser tab.")
    app.run_server(debug=False, port=8050)

In [12]:
if __name__ == "__main__":
    run_dashboard()

Random Walks Dashboard is being created...
Use the 'Update' buttons on each tab to generate visualizations.
Dashboard will open in a new browser tab.
